In [2]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook
# Install tmdbsimple (only need to run once)
!pip install tqdm

In [3]:
# Load API Credentials
with open('/Users/obsid/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['API-Key'], timeout_s=5.0)


In [5]:
# set our API call parameters 
LOCATION = 'Chicago,IL'
TERM = 'Deep Dish Pizza'


In [6]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_IL_pizza.json"
JSON_FILE



'Data/results_in_progress_IL_pizza.json'

In [7]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_IL_pizza.json not found. Saving empty list to file.


In [8]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')



- 0 previous results found.


In [9]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()



dict_keys(['businesses', 'total', 'region'])

In [10]:
## How many results total?
total_results = results['total']
total_results

1800

In [12]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page


20

In [13]:
# Import additional packages for controlling our loop
import time, math
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

90

In [14]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)


In [15]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 


  0%|          | 0/90 [00:00<?, ?it/s]

In [22]:
def create_json_file(JSON_FILE,  delete_if_exists=False):

    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)

# If it DOES exist:
    if file_exists == True:

#Check if user wants to delete if exists
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")

    ## If it does NOT exist:
    else:
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

# CREATE ANY NEEDED FOLDERS
# Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)

# If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [23]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages


[!] Data/results_in_progress_IL_pizza.json already exists. Deleting previous file...
[i] Data/results_in_progress_IL_pizza.json not found. Saving empty list to new file.
- 0 previous results found.


90

In [24]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)


  0%|          | 0/90 [00:00<?, ?it/s]

Exceeded 1000 api calls. Stopping loop.


In [25]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,8vFJH_paXsMocmEO_KAa3w,lou-malnatis-pizzeria-chicago,Lou Malnati's Pizzeria,https://s3-media3.fl.yelpcdn.com/bphoto/FXGmfo...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,7077,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.890357, 'longitude': -87.633704}","[delivery, pickup]",$$,"{'address1': '439 N Wells St', 'address2': '',...",+13128289800,(312) 828-9800,10366.447105
1,Bxd9Y2PsdvIOGIKFQh53RQ,giordanos-chicago-30,Giordano's,https://s3-media1.fl.yelpcdn.com/bphoto/OKxuR9...,False,https://www.yelp.com/biz/giordanos-chicago-30?...,3485,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.88513231411071, 'longitude': -...",[delivery],$$,"{'address1': '130 E Randolph St', 'address2': ...",+13126161200,(312) 616-1200,10689.526750
2,DXwSYgiXqIVNdO9dazel6w,pequods-pizzeria-chicago,Pequod's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/8QJUNb...,False,https://www.yelp.com/biz/pequods-pizzeria-chic...,7524,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.92187, 'longitude': -87.664486}","[delivery, pickup]",$$,"{'address1': '2207 N Clybourn Ave', 'address2'...",+17733271512,(773) 327-1512,11423.025038
3,LrcRLcCGsSNJjTKKK9UgYw,ciprianis-pizza-chicago,Cipriani's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/3FtyP5...,False,https://www.yelp.com/biz/ciprianis-pizza-chica...,137,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.5,"{'latitude': 41.778228, 'longitude': -87.768205}",[delivery],$$,"{'address1': '5840 W 63rd St', 'address2': '',...",+17735823747,(773) 582-3747,6715.230863
4,cxAkuwv7p4gm4Coixw7h9A,lou-malnatis-pizzeria-chicago-9,Lou Malnati's Pizzeria,https://s3-media2.fl.yelpcdn.com/bphoto/vZVADY...,False,https://www.yelp.com/biz/lou-malnatis-pizzeria...,3003,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"{'latitude': 41.871566, 'longitude': -87.627275}","[delivery, pickup]",$$,"{'address1': '805 S State St', 'address2': '',...",+13127861000,(312) 786-1000,9676.008122


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
995,BrKl0XhF3IpCWq34Om_sgg,stefanis-tuscany-café-chicago-3,Stefani's Tuscany Café,https://s3-media4.fl.yelpcdn.com/bphoto/jBEma-...,False,https://www.yelp.com/biz/stefanis-tuscany-caf%...,348,"[{'alias': 'italian', 'title': 'Italian'}]",2.5,"{'latitude': 41.981285, 'longitude': -87.906054}",[],$$,"{'address1': '10000 Bessie Coleman Dr', 'addre...",,,22014.557727
996,gh9MXuna7tDba1arYNjsEA,maggianos-little-italy-oak-brook,Maggiano's Little Italy,https://s3-media1.fl.yelpcdn.com/bphoto/V8KS1a...,False,https://www.yelp.com/biz/maggianos-little-ital...,593,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",3.5,"{'latitude': 41.8499824105333, 'longitude': -8...","[delivery, pickup]",$$,"{'address1': '240 Oakbrook Ctr', 'address2': '...",+16303680300,(630) 368-0300,18318.315880
997,fESy8pfywnwDqJzKpn5GZw,sarpinos-pizzeria-chicago-452,Sarpino's Pizzeria,https://s3-media1.fl.yelpcdn.com/bphoto/390GS4...,False,https://www.yelp.com/biz/sarpinos-pizzeria-chi...,215,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",3.0,"{'latitude': 41.9613, 'longitude': -87.66648}","[delivery, pickup]",$$,"{'address1': '1435 W Montrose St', 'address2':...",+17732811800,(773) 281-1800,15346.641319
998,luu-w6z0cHesP8wNXOb36g,mama-ds-pizza-saint-john,Mama D's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/Vd3oQD...,False,https://www.yelp.com/biz/mama-ds-pizza-saint-j...,22,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 41.44724, 'longitude': -87.4695}",[delivery],$$,"{'address1': '9440 Joliet St', 'address2': '',...",+12193658504,(219) 365-8504,47997.920642
999,DiI2kvTrI_wH3XB6WKFx5A,the-windsor-tavern-and-grill-chicago,The Windsor Tavern and Grill,https://s3-media2.fl.yelpcdn.com/bphoto/TAZEo1...,False,https://www.yelp.com/biz/the-windsor-tavern-an...,92,"[{'alias': 'pubs', 'title': 'Pubs'}]",4.0,"{'latitude': 41.9631, 'longitude': -87.75703}","[delivery, pickup]",$,"{'address1': '4530 N Milwaukee Ave', 'address2...",+17737363400,(773) 736-3400,14738.655116


In [27]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()



2

In [29]:
final_df = final_df.drop_duplicates(subset="id")

In [30]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()


0

In [31]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_CH_deepdishpizza.csv.gz', compression='gzip',index=False)